In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import warnings

In [68]:
books = pd.read_csv("data/clean/books_clean.csv")
ratings = pd.read_csv("data/clean/ratings_clean.csv")
users = pd.read_csv("data/clean/users_clean.csv")
books_users_ratings = pd.read_csv("data/clean/books_users_ratings.csv")

**Recommendations based on correlations**

In [31]:
#dropping books with one review to reduce size of dataset
average_ratings = pd.DataFrame(books_users_ratings.groupby('unique_isbn')['book_rating'].mean())
average_ratings['rating_count'] = pd.DataFrame(books_users_ratings.groupby('unique_isbn')['book_rating'].count())
average_ratings.sort_values('rating_count', ascending=False).head()
average_ratings['greater1'] = (average_ratings.rating_count > 9)

In [32]:
average_ratings.head()

,book_rating,rating_count,greater1
unique_isbn,,,
0000913154,8.0,1,False
000104687X,6.0,1,False
0001047213,9.0,1,False
000104799X,7.5,2,False
0001061127,8.0,1,False


In [33]:
books_users_ratings['dropornot'] = [average_ratings.greater1[x] == False for x in books_users_ratings.unique_isbn]

In [34]:
reduced_books_users_ratings = books_users_ratings[books_users_ratings.dropornot == False]

In [35]:
# dropping users with one review
userdrop = reduced_books_users_ratings.groupby('user_id')['book_rating'].count() <= 2
reduced_books_users_ratings['dropuser'] = [userdrop[x] for x in reduced_books_users_ratings['user_id']]

/Users/joshuagoldberg/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
reduced_books_users_ratings = pd.merge(reduced_books_users_ratings, users, on="user_id")
reduced_books_users_ratings.head()

,user_id,isbn,book_rating,book_title,book_author,year_of_publication,unique_isbn,dropornot,dropuser,age,city,state,country
0,276744,038550120X,7,a painted house,john grisham,2001,044023722X,False,True,34.786876,torrance,california,usa
1,11676,038550120X,10,a painted house,john grisham,2001,044023722X,False,False,34.786876,NaN,n/a,n/a
2,11676,044023722X,8,a painted house,john grisham,2001,044023722X,False,False,34.786876,NaN,n/a,n/a
3,11676,0671537458,8,waiting to exhale,terry mcmillan,1992,0671537458,False,False,34.786876,NaN,n/a,n/a
4,11676,0671864173,5,waiting to exhale,terry mcmillan,1993,0671537458,False,False,34.786876,NaN,n/a,n/a


In [37]:
reduced_books_users_ratings = reduced_books_users_ratings[reduced_books_users_ratings.dropuser == False]

In [38]:
reduced_books_users_ratings.to_csv("data/clean/reduced_books_users_ratings.csv", index=False)

In [69]:
pd.read_csv("data/clean/reduced_books_users_ratings.csv").shape

(110199, 13)

In [79]:
len(users.user_id.unique())

277332

In [39]:
user_item_rating = reduced_books_users_ratings[['user_id', 'unique_isbn', 'book_rating']]
user_item_rating.head()

,user_id,unique_isbn,book_rating
1,11676,044023722X,10
2,11676,044023722X,8
3,11676,0671537458,8
4,11676,0671537458,5
5,11676,0679776818,8


In [40]:
# dividing into test and train
from sklearn import model_selection
train_data, test_data = model_selection.train_test_split(user_item_rating, test_size=0.40)

In [41]:
train_users = len(train_data['user_id'].unique())
train_books = len(train_data['unique_isbn'].unique()) 

In [42]:
train_users

10265

In [43]:
train_books

5152

In [44]:
test_users = len(test_data['user_id'].unique())
test_books = len(test_data['unique_isbn'].unique())

In [45]:
test_users

9546

In [46]:
test_books

5101

In [47]:
#creating a user-item matrix to calculate similarity between users


In [48]:
train_matrix = np.zeros((train_users,train_books))

In [49]:
#Map the user_id and unique_isbn fields to sequential integers for matrix processing
u_unique_train = train_data.user_id.unique()  # create a 'set' (i.e. all unique) list of vals
train_data_user2idx = {o:i for i, o in enumerate(u_unique_train)}
# Get int mapping for unique_isbn
b_unique_train = train_data.unique_isbn.unique()  # create a 'set' (i.e. all unique) list of vals
train_data_book2idx= {o:i for i, o in enumerate(b_unique_train)}

### TESTING SET
# Get int mapping for user_id
u_unique_test = test_data.user_id.unique()  # create a 'set' (i.e. all unique) list of vals
test_data_user2idx = {o:i for i, o in enumerate(u_unique_test)}
# Get int mapping for unique_isbn
b_unique_test = test_data.unique_isbn.unique()  # create a 'set' (i.e. all unique) list of vals
test_data_book2idx = {o:i for i, o in enumerate(b_unique_test)}

In [50]:
train_data['u_unique'] = train_data['user_id'].map(train_data_user2idx)
train_data['b_unique'] = train_data['unique_isbn'].map(train_data_book2idx)

### TESTING SET
test_data['u_unique'] = test_data['user_id'].map(test_data_user2idx)
test_data['b_unique'] = test_data['unique_isbn'].map(test_data_book2idx)



/Users/joshuagoldberg/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/joshuagoldberg/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/joshuagoldberg/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [51]:
### Convert back to 3-column df
train_data_new = train_data[['u_unique', 'b_unique', 'book_rating']]
test_data_new = test_data[['u_unique', 'b_unique', 'book_rating']]

In [52]:
for entry in train_data_new.itertuples(): # entry[1] is the user-id, entry[2] is the book-isbn
    train_matrix[entry[1]-1, entry[2]-1] = entry[3] # -1 is to counter 0-based indexing

In [53]:
train_matrix

array([[9., 0., 0., ..., 0., 0., 0.],
       [0., 8., 0., ..., 0., 0., 0.],
       [0., 0., 7., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 7.]])

In [54]:
#Calculating cosine similarity
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_matrix, metric='cosine')
item_similarity = pairwise_distances(train_matrix.T, metric='cosine') # .T transposes the matrix (NumPy)

In [55]:
def predict(ratings, similarity, type='user'): # default type is 'user'
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        # Use np.newaxis so that mean_user_rating has the same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [56]:
#predicts ratings for every book and every user
item_prediction = predict(train_matrix, item_similarity, type='item')
user_prediction = predict(train_matrix, user_similarity, type='user')

In [57]:
#function to get top ten recommendations for a given user id based on user similarity scores
def predict_user(user_id, user_prediction, books_users_ratings):
    uniqueuser = train_data_user2idx[user_id]
    ranks = pd.DataFrame(user_prediction[uniqueuser])
    ranks['uniquebook'] = [i for i,o in enumerate(ranks[0])]
    ranks.columns = ['rating', 'uniquebook']
    ranks= ranks.sort_values(by=['rating'],ascending=False)
    isbn = []
    for i,v in enumerate(ranks.uniquebook):
        for key, value in train_data_book2idx.items():
            if v == value:
                isbn.append(key)
    ranks['unique_isbn'] = isbn
    ranks['rated'] = [1 if x in reduced_books_users_ratings[reduced_books_users_ratings.user_id == user_id].unique_isbn.tolist() else 0 for x in ranks.unique_isbn]
    ranks = ranks[ranks.rated==0]
    ranks=ranks.head(10)
    recommendations = [books_users_ratings[books_users_ratings.unique_isbn==x].book_title.tolist()[1] for x in ranks.unique_isbn]
    ranks['titles'] =recommendations
    print(ranks)

In [58]:
#example prediction of ten books for a given user based on user similarity scores
predict_user(161882,user_prediction,books_users_ratings)

       rating  uniquebook unique_isbn  rated  \
72   0.221426          72  0380750872      0   
45   0.191304          45  0060507845      0   
395  0.146723         395  0679876154      0   
337  0.136522         337  0060932759      0   
109  0.135965         109  0553102338      0   
348  0.124843         348  0312990456      0   
2    0.123481           2  0671034006      0   
448  0.123304         448  0374525188      0   
121  0.121687         121  0060930934      0   
674  0.119397         674  0679737383      0   

                                         titles  
72                                  secret fire  
45        a promising man (and about time, too)  
395                            the tin princess  
337                         daughter of fortune  
109                          walking to mercury  
348  one for the money (a stephanie plum novel)  
2                                  heartbreaker  
448                                housekeeping  
121   wasted : a memo

In [65]:
#function to get top ten recommendations for a given user id based on item similarity scores
def predict_user(user_id, item_prediction, books_users_ratings):
    uniqueuser = train_data_user2idx[user_id]
    ranks = pd.DataFrame(item_prediction[uniqueuser])
    ranks['uniquebook'] = [i for i,o in enumerate(ranks[0])]
    ranks.columns = ['rating', 'uniquebook']
    ranks= ranks.sort_values(by=['rating'],ascending=False)
    isbn = []
    for i,v in enumerate(ranks.uniquebook):
        for key, value in train_data_book2idx.items():
            if v == value:
                isbn.append(key)
    ranks['unique_isbn'] = isbn
    ranks['rated'] = [1 if x in reduced_books_users_ratings[reduced_books_users_ratings.user_id == user_id].unique_isbn.tolist() else 0 for x in ranks.unique_isbn]
    ranks = ranks[ranks.rated==0]
    ranks=ranks.head(10)
    recommendations = [books_users_ratings[books_users_ratings.unique_isbn==x].book_title.tolist()[1] for x in ranks.unique_isbn]
    ranks['titles'] =recommendations
    return(ranks)

In [66]:
#example prediction of ten books for a given user based on item similarity scores
predict_user(161882,item_prediction,books_users_ratings)

,rating,uniquebook,unique_isbn,rated,titles
4996,0.013782,4996,0446600709,0,assumed identity
4955,0.013782,4955,0449002985,0,your oasis on flame lake (ballantine reader's ...
5143,0.013782,5143,0449148246,0,day of atonement (peter decker &amp; rina laza...
5036,0.013782,5036,0380698439,0,behind the attic wall (avon camelot books (pap...
4016,0.013782,4016,0440237424,0,the best revenge
5089,0.013782,5089,038549744X,0,english passengers
4950,0.013632,4950,0963270702,0,the satanic verses
4406,0.013603,4406,0345334302,0,ringworld engineers
5043,0.013595,5043,0553278398,0,prelude to foundation (foundation novels (pape...
793,0.013582,793,0786863218,0,"don't worry, make money: spiritual and practic..."


In [61]:
### Testing the prediction methods bases on rmse
# Create user-item matrices
n_users = test_data['u_unique'].nunique()
n_books = test_data['b_unique'].nunique()

# First, create an empty matrix of size USERS x BOOKS (this speeds up the later steps)
test_matrix = np.zeros((n_users, n_books))


In [62]:
test_data.head()

,user_id,unique_isbn,book_rating,u_unique,b_unique
122598,80375,0312978367,8,0,0
75414,93046,0345392825,10,1,1
5210,64310,0451191706,9,2,2
94819,274466,0140186409,10,3,3
49026,105374,0679721037,8,4,4


In [63]:

# Then, add the appropriate vals to the matrix by extracting them from the df with itertuples
for entry in test_data_new.itertuples(): # entry[1] is the user-id, entry[2] is the book-isbn
    test_matrix[entry[1]-1, entry[2]-1] = entry[3] # -1 is to counter 0-based indexing

In [64]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(prediction, test_matrix):
    prediction = prediction[test_matrix.nonzero()].flatten()
    test_matrix = test_matrix[test_matrix.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, test_matrix))

# Call on test set to get error from each approach ('user' or 'item')
print(f'User-based CF RMSE: {rmse(user_prediction, test_matrix)}')
print(f'Item-based CF RMSE: {rmse(item_prediction, test_matrix)}')

User-based CF RMSE: 7.978908822282232
Item-based CF RMSE: 7.981911499040296
